<a href="https://colab.research.google.com/github/mharrisonbaker/samplework/blob/master/llm_negative_treatment_baker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libraries

In [109]:
!pip install openai

In [110]:
!pip install tiktoken

In [111]:
import requests
from bs4 import BeautifulSoup
import openai
from getpass import getpass
import os
import textwrap


 # enter api key

In [172]:
#enter your API key here
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')

Enter your OpenAI API key: ··········


In [161]:
openai.api_key = os.environ['OPENAI_API_KEY']

## insert test slug here

In [149]:
slug1 = 'littlejohn-v-state-7'
slug2 = 'beattie-v-beattie'
slug3 = 'travelers-indem-co-v-lake'
slug4 = 'tilden-v-state'
slug5 = 'in-re-lee-342013'
#slug6=set_aside_slug

slug = slug4 #change this line to test slugs

# define functions

In [168]:
def get_case_url(slug):
    url = f"https://casetext.com/api/search-api/doc/{slug}/html"
    return url

In [150]:
def retrieve_case_text(slug):
    url = f"https://casetext.com/api/search-api/doc/{slug}/html"
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(response.content, 'html.parser')
        case_text = soup.get_text()
        return case_text
    else:
        print(f"Failed to retrieve case: {slug}")
        return None


In [142]:
# retrieve_case_text(slug=test_slug)

In [162]:
def extract_negative_treatments(slug):
    # Fetch the case text using the slug
    case_text = retrieve_case_text(slug)

    # Break the case text into manageable parts
    case_text_parts = textwrap.wrap(case_text, width=12000)  # 12000 characters per part


    # Process each part using the LLM
    treatments = []
    for part in case_text_parts:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a lawyer."},
                {"role": "user", "content": 'Given the following legal opinion:\n\n'+part+'\n\nIdentify any phrases or sections indicating that a prior case was overruled or that its holding was limited.'},
            ],
            temperature=0.4
        )

        # Save the LLM's response
        treatments.append(response['choices'][0]['message']['content'])

    # Remove duplicates while preserving order
    treatments = list(dict.fromkeys(treatments))

    return treatments

#use function

In [171]:
# Use the function
treatments = extract_negative_treatments(slug=slug)
print(treatments)
url = get_case_url(slug)
print(url)

['There are no phrases or sections in the legal opinion indicating that a prior case was overruled or that its holding was limited.', "Based on the provided legal opinion, there are no phrases or sections indicating that a prior case was overruled or that its holding was limited. The opinion mentions the State's request to overrule a previous holding in Johnson but states that the court declines to expressly overrule it. However, the opinion does not mention any specific phrases or sections indicating that a prior case was overruled or limited."]
https://casetext.com/api/search-api/doc/tilden-v-state/html
